This script is a spin-off of the tutorial avaiable here:

https://mailtrap.io/blog/python-send-email-gmail/

I also had to some annoying google credentialing

Summary of Key Steps:
Be your own google admin
Enable 2FA on your Gmail account.
Generate an app-specific password via Google’s App Passwords page.
Use the app-specific password in your Python script to authenticate and send emails.

In [2]:
#Pip installs may be necessary if you receive module not found errors
import smtplib
import pandas as pd
import requests
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from io import StringIO

In [3]:
# Step 1: Fetch survey data from the Google Sheets URL

# Example input: spreadsheet_id and tab_id (gid)

spreadsheet_id = '1iDsbDwcxj5nygYZASAIIlY7Jo58tzrxK4qc4Xemj6vU'  # Example spreadsheet ID
tab_id = '0'  # Example tab ID (gid)
csv_url = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={tab_id}'

# Notes:
# - The 'spreadsheet_id' is the unique identifier of the Google Sheet, find after '/d/' and before '/edit'.# 
# - The 'tab_id' (also known as gid) is the identifier of the specific sheet/tab , found after 'gid='.
#   Example URL: 'https://docs.google.com/spreadsheets/d/<spreadsheet_id>/edit#gid=<tab_id>'

response_survey = requests.get(csv_url)
survey_data = response_survey.content
survey_df = pd.read_csv(StringIO(survey_data.decode('utf-8')))

#Credentials 
pw = input("Enter the password Kevin gave you in class to continue ")

Survey data loaded successfully.


Enter the password Kevin gave you in class  jnts tvtq sgik zwkh


In [4]:
# Step 2: Email details and setup
# this is a bit of a burner email. Again, you could totally hack me, but you wouldn't really be able to get alot
from_email = 'dab_class@hmsdataconsulting.com'

#enter your email here:
to_email = 'kevinpatrickbradley@gmail.com'
subject = "Escape Room"
body = "Hi Kevin,\n\nPlease find the attached survey data. I am leaving now. \n\nBest regards,\nYour Name"

# Step 3: Create the email
msg = MIMEMultipart()
#MIME means Multipurpose Internet Mail Extensions (MIME) 
# basically an object that goes over mail is very unique in structure. 
#MIME helps sort headers into the spot they are supposed to be, and body into the right parts

msg['From'] = from_email
msg['To'] = to_email
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

# Step 4: Attach the CSV data
csv_buffer = StringIO()
survey_df.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)

part = MIMEBase('application', 'octet-stream')
#This is the kind of thing I don't worry about too much in a script I take from a turorial 
#Possibly I will need to worry about it if there is troubleshooting necessary 

part.set_payload(csv_buffer.getvalue())
encoders.encode_base64(part)
part.add_header('Content-Disposition', f'attachment; filename="survey_data.csv"')
msg.attach(part)

#print statements help because this is going to be an executable 
print("Email with attachment prepared.")

Email with attachment prepared.


In [5]:
# Step 5: Set up SMTP and send the email
# this means Simple Mail Transfer Protocol

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()

# Login to the Gmail account (make sure to use an app-specific password if using 2FA)
server.login(from_email, pw)  

# Convert the message to a string and send
text = msg.as_string()
server.sendmail(from_email, to_email, text)
server.quit()
print(f"Email successfully sent to {to_email}.")

Email successfully sent to kevinpatrickbradley@gmail.com.
